# NMF / sparse matrix pair

This pair problem covers a few techniques: some specfic to dealing with sparse matrices, and some relating to using NMF to look at the top words for given topics.

## Data

We'll be using the BBC dataset. These are articles collected from 5 different topics, with the data pre-processed. 

You can download the data by clicking [here](http://mlg.ucd.ie/files/datasets/bbc.zip). **Save the data and unzip it in the same folder as this notebook!** The data consists of a few files. The steps we'll be following are:

* *bbc.terms* is just a list of words 
* *bbc.docs* is a list of artcles listed by topic.

At a high level, we're going to 

1. Turn the `bbc.mtx` file into a sparse matrix.
1. Decompose that sparse matrix using NMF.
1. Use the resulting components of NMF to analyze the topics that result.

## Read in the data

You can read in the data with these lines:

In [ ]:
with open('bbc.mtx') as f:
    content = f.readlines()

In [ ]:
content.pop(0)
content.pop(0)

## Exercise

Turn this into a list of tuples representing a sparse matrix. Remember the description of the file from above:

* *bbc.mtx* is a list: first column is **wordID**, second is **articleID** and the third is the number of times that word appeared in that article.

So, if word 1 appears in article 3, 2 times, one element of your list will be:

`(1, 3, 2)`

In [ ]:
sparsemat = [tuple(map(int,map(float,c.split()))) for c in content]
sparsemat

## Turn this into an array

Now, we're going to learn a cool function to turn sparse matrix objects like this into 2D arrays. Check out the [coo matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) documentation and the example below:

In [ ]:
from scipy.sparse import coo_matrix
import numpy as np
row  = np.array([0, 3, 1, 0])
col  = np.array([0, 3, 1, 2])
data = np.array([4, 5, 7, 9])
coo_matrix((data, (row, col)), shape=(4, 4)).toarray()

## Exercise:

Turn the sparse matrix object you created above into an array using `coo_matrix` and `.toarray()`.

In [ ]:
from scipy.sparse import coo_matrix
rows = [x[0] for x in sparsemat]
cols = [x[1] for x in sparsemat]
values = [x[2] for x in sparsemat]
coo = coo_matrix((values, (rows, cols)))

## NMF

NMF is a way of decomposing a matrix of documents and words so that one of the matrices can be interpreted as the "loadings" or "weights" of each word on a topic. 

Check out [the NMF documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) and the [examples of topic extraction using NMF and LDA](http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html).

## Exercise

Import `NMF`, define a model object with 5 components, and `fit_transform` the data you created above.

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=818)
doc_topic = model.fit_transform(coo)

doc_topic.shape
# we should have 9636 observations (articles) and five latent features

In [ ]:
# find feature with highest value per doc
np.argmax(doc_topic, axis=1)

## Exercise: 

Check out the `components` of this model:

In [ ]:
model.components_.shape

This is five rows, each of which is a "topic" containing the weights of each word on that topic. The exercise is to _get a list of the top 10 words for each topic_. You can just store this in a list of lists.

**Note:** Just like you read in the data above, you'll have to read in the words from the `bbc.terms` file.

In [ ]:
with open('bbc.terms') as f:
    content = f.readlines()
words = [c.split()[0] for c in content]

In [ ]:
topic_words = []
for r in model.components_:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:12]
    topic_words.append([words[e[1]] for e in a])

In [ ]:
topic_words

The original data had 5 topics, as listed in `bbc.docs`. 

```
Business
Entertainment
Politics
Sport
Tech
```

In "real life", we would have found a way to use these to inform the model. But for this little demo, we can just compare the recovered topics to the original ones. And they seem to match reasonably well. The order is different, which is to be expected in this kind of model.

In [ ]:
with open('bbc.docs') as d:
    content = d.readlines()
    
content